<a href="https://colab.research.google.com/github/SJa-638/test-robot/blob/main/Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **To Download the needed Libaries Exceute the following Commands**

In [ ]:

!pip install -q keras

In [ ]:
!pip install kaggle

In [ ]:
!pip install opencv-python

In [ ]:
!apt install git

## **Connecting to The Google drive to get The dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

[Garabage Clssification Dataset](https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification/data)

## Loading Libraries

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import random
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import random
import shutil

## Tensorflow
[TensorFlow For Beginnera](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb)
*VERY IMP NOTE ON DOWNLOADING TENSORFLOW ON YOUR LOCAL COMPUTER BE CAREFUL ABOUT USING PIP TO INSTALL IT AND MAKE SURE VERSIONS OF PYTHON TENSORFLOW AND PIP ARE RIGHT OTHERWISE IT WON'T BE RECOGNIZED BY THE COMPUTER / OR DOWNLOAD IT MANUALLY FOLLOWING A TUTORIAL/ OR USE IT ON GOOGLE COLAB IT MAINTAINS THESE ERRORS*
1. **Collect a dataset of images of garbage, labeled with the type of garbage.** You can either collect this dataset yourself or use a pre-existing dataset. Some pre-existing datasets that you can use include:
    - TrashNet
    - RecycleNet
    - Google AI CircularNet dataset
2. **Split the dataset into training and test sets.** The training set will be used to train the model, and the test set will be used to evaluate the performance of the model on unseen data. A common split is to use 80% of the dataset for training and 20% of the dataset for testing.
3. **Preprocess the images.** This may involve resizing the images to a consistent size, normalizing the pixel values, and/or cropping the images to focus on the garbage item.
4. **Choose a model architecture.** There are a number of different TensorFlow model architectures that you can use for garbage classification. Some popular choices include:
    - ResNet
    - MobileNet
    - EfficientNet
    - VGG16
5. **Compile the model.** This involves specifying the loss function, optimizer, and learning rate.
6. **Train the model.** This involves feeding the training images to the model and updating the model parameters to minimize the loss function.
7. **Evaluate the model.** This involves feeding the test images to the model and measuring the accuracy of the model's predictions.
8. **Save the model.** Once you are satisfied with the performance of the model, you can save it to a file. This will allow you to load the model later and use it to classify new images of garbage.

## Tutorial For tensorflow
[link text](https://www.youtube.com/watch?v=CMu25mtyl-s)

In [ ]:
dataset = '/content/drive/MyDrive/dataset/Garbage_classification/Garbage_classification'
trainGarabge = '/content/drive/MyDrive/dataset/Garbage_classification/Garbage_classification'
valdData = '/content/drive/MyDrive/dataset/Garbage_classification/Garbage_classification'
# Navigate to the directory where the dataset is uploaded
os.chdir(trainGarabge)
os.chdir(valdData)

dataset = tf.keras.preprocessing.image_dataset_from_directory(dataset)
# Load the dataset into Keras
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    trainGarabge,
    seed=123,
    validation_split=0.2,
    subset="training",
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    valdData,
    validation_split=0.2,
    subset="validation",
    seed=123,
)

Found 2527 files belonging to 6 classes.
Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.


In [ ]:
import tensorflow_datasets as tds

# Defining batch size and image labels

batch_size = 64
dataset_name = dataset
class_names = dataset.class_names
print(class_names)

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [ ]:
size = (512,512)
#TO be Continued
ds_train = trainGarabge.map(lambda image, label:(tf.image.resize(image,size),label)
ds_vald = valdData.map(lambda image, label:(tf.image.resize(image,size),label)


In [ ]:
# Create the model
model = tf.keras.Sequential()


# Add convolutional layers
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers
model.add(tf.keras.layers.Flatten())

# Add dense layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Train the model
# TO DO
model.fit(trainGarabge,'garbage',epochs=10,validation_data=test_dataset)



# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)

# Save the model
model.save('garbage_model.h5')

# Load the trained TensorFlow model
model = tf.keras.models.load_model('garbage_classifier.h5')



Found 2527 files belonging to 6 classes.
Found 2527 files belonging to 6 classes.


IndexError: ignored

In [ ]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json


In [ ]:
!sudo chown root:root ~/.kaggle
!sudo chown root:root ~/.kaggle/kaggle.json

In [ ]:
!sudo chmod 600 ~/.kaggle
!sudo chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kaggle
# Set up the Kaggle API
api = KaggleApi()
api.authenticate()

# Path to the downloaded dataset
dataset_path = 'D:\Kaggle dataset archive.zip'  # Path to the extracted dataset files

# Download the Kaggle dataset
dataset_name = 'lkihjh777'  # Replace with the name of the Kaggle dataset you want to download
#api.dataset_download_files(dataset_name, path=dataset_path, unzip=True)

# Iterate over the downloaded images
for file_name in os.listdir(dataset_path):
    image_path = os.path.join(dataset_path, file_name)

    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Perform image processing operations on the image as needed
    # Example: Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Example: Apply a threshold to create a binary image
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

    # Perform further processing or analysis on the image

    # Display the processed image
    cv2.imshow('Processed Image', binary_image)
    cv2.waitKey(0)

    # Clean up resources
    cv2.destroyAllWindows()

ValueError: ignored

## **Capturing image from webcam to be classified to the By the trained model**




In [ ]:
# Capture an image from the webcam
cap = cv2.VideoCapture(0)
ret, frame = cap.read()


In [ ]:

# Preprocess the image
image = cv2.resize(frame, (224, 224))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = np.expand_dims(image, axis=0)

# Classify the image using the trained model
prediction = model.predict(image)
predicted_class = np.argmax(prediction)

# Print the predicted class
print("Predicted class:", predicted_class)

# Release the webcam
cap.release()

## Upload to Github

In [5]:
!git clone https://github.com/SJa-638/test-robot.git

Cloning into 'test-robot'...
remote: Enumerating objects: 2570, done.
remote: Total 2570 (delta 0), reused 0 (delta 0), pack-reused 2570
Receiving objects: 100% (2570/2570), 59.33 MiB | 18.98 MiB/s, done.
Resolving deltas: 100% (166/166), done.
Updating files: 100% (2316/2316), done.


In [6]:
!git clone https://github.com/SJa-638/test-robot.git

fatal: destination path 'test-robot' already exists and is not an empty directory.


In [13]:
!git config --global user.name "SJa-638"
!git config --global user.email "sazab638@gmail.com"


# Navigate to the cloned repository directory
%cd '/content/test-robot'

# Upload the notebook to the cloned repository
!git add .
!git commit -m "Add notebook"

# Push the changes to the remote repository
!git push

/content/test-robot
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
